## Outline
* What is JSON
* AESON
* Example
* Extra

In this lecture, we will discuss the implementation of JSON in Haskell. The structure and sections are listed above. First, we will touch on what JSON is and how it is defined. After that, we will have a look how Haskell implements this. Lastly, we will give some extracurricular info on CBOR, which is similar to JSON.

### What is JSON
JSON is an abbreviation of **J**ava**S**cript **O**bject **N**otation, and it is a file format that is designed to interchange information in a human-readable form. The origin of JSON lies in the JavaScript language, this is why the syntax it uses looks similar to that language. The representation of data and its syntax follow these 4 rules

Rule | Description | Example |
--- | --- | --- | 
1 | Data is in key/value pairs | `"firstName" : "Bob"`
2 | Data is separated by commas | `"firstName" : "Bob", "lastName" : "Smith"` 
3 | Curly brackets `{}` group objects in an unordered set | `{"one" : 1, "two" : 2, "two" : 3}`
4 | Square brackets `[]` hold arrays in an ordered collection| `"numbers": ["one", "two", "three"]`

The key value is always a string in JSON, this while the values can be of the types

Type | Examples
--- | --- |
String | `"name" : "Alice"`
Number | `{"one" : 210, "two" : -210, "three" : 3.14, "four" : 1.0E+2}`
Boolean | `"Haskell is cool" : true`
Null | `"key" : null `
Object | `{"favorite food" : "pasta"}`
Array | `[1,2,3]`

JSON is handy when structured data needs to be made visible. It is often used in API's and for computer-human interactions.


### AESON
The library that can parse and encode Haskell types to the JSON standard is called Aeson [(1)](https://hackage.haskell.org/package/aeson). This library is named after Aeson, the father of Jason, in Greek mythology. Aeson is not installed by default, since it is an external library. For the current notebook, it can be installed via the execution of



In [ ]:
:!stack build aeson

Like in the above JSON standard, the Aeson has the following possible values.
```haskell
-- | A JSON value represented as a Haskell value.
data Value = Object Object
           | Array Array         --- In JSON []
           | String Text         --- In JSON ""
           | Number Scientific   --- In JSON 3.1415E+0
           | Bool Bool           --- In JSON true/false
           | Null                --- In JSON null
```

More explicitly, we have that we can encode these to JSON via the function `encode`

In [22]:
{-# LANGUAGE OverloadedStrings #-}
import Data.Aeson

:t encode

encode [1,3,4]
encode "Hello World"
encode 3.1415
encode False
encode Null

encode :: forall a. ToJSON a => a -> ByteString

"[1,3,4]"

"\"Hello World\""

"3.1415"

"false"

"null"

To convert these back, we can use the `decode` function. Notice that the type signature of this function is always to a `Maybe a`. This is because decoding a `ByteString` that could be in the JSON format might fail. Decoding the above examples gives

In [23]:
{-# LANGUAGE OverloadedStrings #-}
import Data.Aeson

:t decode

decode "[1,3,4]" :: Maybe [Int]
decode "\"Hello World\"" :: Maybe String
decode "3.1415" :: Maybe Double
decode "false" :: Maybe Bool
decode "null" :: Maybe Value

decode :: forall a. FromJSON a => ByteString -> Maybe a

Just [1,3,4]

Just "Hello World"

Just 3.1415

Just False

Just Null

The `null` string is decoded as a `Maybe Value` since the Null is not a native object in Haskell, it is defined only in the data type `Value` given above.

### Example
If we have the data type 

In [7]:
data Person = Person { firstName :: String, lastName :: String, age :: Int}

Haskell can derive from the type structure a general conversion into the JSON format. Here the type constructors are replaced with the key values (as strings), followed by the information in the fields. To use this automated way of encoding Haskell types into JSON, you need the following things

- Import `import GHC.Generics`
- Use the pragma `{-# LANGUAGE DeriveGeneric  #-}`
- Use the pragma `{-# LANGUAGE DeriveAnyClass #-}`
- Use the pragma `{-# LANGUAGE OverloadedStrings #-}`
- Derive the instances of `Generic` 
- Derive the instances of `ToJSON`  and `FromJSON` for your type

These things are necesairy for Haskell to derive 

In [21]:
{-# LANGUAGE DeriveGeneric #-}
{-# LANGUAGE DeriveAnyClass #-}
{-# LANGUAGE OverloadedStrings #-}

import Data.Aeson
import GHC.Generics
import Data.Maybe

data Person = Person { firstName :: String, lastName :: String, age :: Int} deriving (Generic, Show, ToJSON, FromJSON)

johnDoe = Person {firstName = "John", lastName = "Doe", age = 31}

encode johnDoe
tomHanks = decode "{\"age\":66,\"lastName\":\"Hanks\",\"firstName\":\"Tom\"}" :: Maybe Person

print $ fromJust tomHanks

"{\"age\":31,\"lastName\":\"Doe\",\"firstName\":\"John\"}"

Person {firstName = "Tom", lastName = "Hanks", age = 66}

### Extra
The following information is extra and is informational only. In this lesson, we discussed the file format JSON that is designed to interchange information in a human-readable form. But in lesson 12 we learned that it is sometimes faster to use binary representation of things when we are processing them with a computer. The JSON standard still uses a lot of overhead to manage the data that it is trying to communicate. If we do not care about the human readability of the data, we can use the CBOR standard instead of the JSON standard. This stands for **C**oncise **B**inary **O**bject **R**epresentation and does the same as JSON but in a faster and more compressed way.

